<a href="https://colab.research.google.com/github/portneon/Fleet_maintainance_prediction/blob/main/randomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
import sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import joblib

In [22]:
print("numpy",np.__version__)
print("pandas",pd.__version__)
print("sklearn",sklearn.__version__)


numpy 2.0.2
pandas 2.2.2
sklearn 1.6.1


In [23]:
df = pd.read_csv('/content/ai4i2020.csv')


In [24]:
df


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


In [25]:
df = df.drop(columns= ['UDI','Product ID'])

In [26]:
df["Torque [Nm]"] = pd.to_numeric(df["Torque [Nm]"], errors="coerce")
df = pd.get_dummies(df, columns=["Type"], drop_first=True)

In [27]:
df

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,False,True
1,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,True,False
2,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,True,False
3,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,True,False
4,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0,0,0,0,0,0,False,True
9996,298.9,308.4,1632,31.8,17,0,0,0,0,0,0,False,False
9997,299.0,308.6,1645,33.4,22,0,0,0,0,0,0,False,True
9998,299.0,308.7,1408,48.5,25,0,0,0,0,0,0,False,False


In [28]:
df.rename(columns={
    'Air temperature [K]': 'Air_temperature',
    'Process temperature [K]': 'Process_temperature',
    'Rotational speed [rpm]': 'Rotational_speed',
    'Torque [Nm]': 'Torque',
    'Tool wear [min]': 'Tool_wear'}, inplace=True)


In [29]:
X = df.drop(columns=[
    "Machine failure",
    "TWF", "HDF", "PWF", "OSF", "RNF"
])

y = df["Machine failure"]


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,stratify=y,random_state=42)

In [31]:
X_train

,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Type_L,Type_M
4058,302.0,310.9,1456,47.2,54,False,True
1221,297.0,308.3,1399,46.4,132,False,True
6895,301.0,311.6,1357,45.6,137,False,True
9863,298.9,309.8,1411,56.3,84,True,False
8711,297.1,308.5,1733,28.7,50,True,False
...,...,...,...,...,...,...,...
980,296.1,306.7,1409,42.8,134,True,False
4266,302.7,311.1,1440,39.5,146,True,False
7772,300.3,311.5,1464,41.0,29,False,False
5780,301.7,311.2,1517,42.4,113,True,False


In [32]:
rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
THRESHOLD = 0.3

y_prob = rf.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= THRESHOLD).astype(int)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.95      0.97      1932
           1       0.37      0.91      0.53        68

    accuracy                           0.94      2000
   macro avg       0.68      0.93      0.75      2000
weighted avg       0.98      0.94      0.96      2000



In [33]:
joblib.dump(
    {
        "model": rf,
        "threshold": THRESHOLD,
        "features": X_train.columns.tolist()
    },
    "stage1_failure_detector.pkl"
)


['stage1_failure_detector.pkl']

**NOW TRANING THE PART TO PREDICT THE FALIURE CLASSES OF THE MACHINE WE ARE ONLY CONSIDERING DATA WITH FALIURE**

In [34]:
failure_cols = ["TWF", "HDF", "PWF", "OSF"]

In [35]:
X_stage2 = X_train[y_train == 1]
Y_stage2 = df.loc[X_stage2.index, failure_cols]

In [40]:
X_s2_train, X_s2_val, Y_s2_train, Y_s2_val = train_test_split(X_stage2,Y_stage2,test_size=0.2,random_state=42)

In [41]:
X_s2_train

,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Type_L,Type_M
2749,299.7,309.2,1685,28.9,179,False,True
4391,302.2,310.1,1368,51.6,12,False,True
4441,302.5,310.2,1329,50.6,148,True,False
1808,298.1,307.7,2567,12.8,125,True,False
4282,301.9,309.9,1374,48.5,180,True,False
...,...,...,...,...,...,...,...
1496,298.0,308.7,1268,69.4,189,True,False
4642,303.2,311.4,1238,54.6,226,True,False
3140,300.4,309.9,2563,12.8,81,False,False
3943,302.3,311.4,1333,66.7,205,True,False


In [37]:
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier

xgb = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

stage2_model = MultiOutputClassifier(xgb)
stage2_model.fit(X_s2_train, Y_s2_train)


MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.8, device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss',
                                              feature_types=None,
                                              feature_weights=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.05, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=4,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=300, n_jobs=-1,
                                              num_parallel_tree=None, ...))

In [38]:
probs = stage2_model.predict_proba(X_s2_val)


In [39]:
joblib.dump({
    "stage1_threshold": 0.3,
    "failure_thresholds": FAILURE_THRESHOLDS,
    "failure_cols": failure_cols,
    "feature_columns": X_train.columns.tolist()
}, "config.pkl")

NameError: name 'FAILURE_THRESHOLDS' is not defined

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

thresholds = np.arange(0.1, 0.9, 0.1)

for i, col in enumerate(failure_cols):
    print(f"\n🔍 Tuning for {col}")
    y_true = Y_stage2[col].values
    y_prob = probs[i][:, 1]

    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        recall = ( (y_pred & y_true).sum() / y_true.sum() ) if y_true.sum() > 0 else 0
        print(f"Threshold {t:.1f} → Recall: {recall:.2f}")


In [ ]:
FAILURE_THRESHOLDS = {
    "TWF": 0.5,   # Tool Wear Failure
    "HDF": 0.5,   # Heat Dissipation Failure
    "PWF": 0.5,   # Power Failure
    "OSF": 0.5    # Overstrain Failure
}


In [ ]:
import joblib
joblib.dump(stage2_model, "stage2_failure_classifier.pkl")

# Save config too
joblib.dump({
    "stage1_threshold": 0.3,
    "failure_thresholds": FAILURE_THRESHOLDS,
    "failure_cols": failure_cols
}, "config.pkl")



In [ ]:
import xgboost
print("sklearn",xgboost.__version__)